#USE AI AGENTs, like AUTOGPT or BABYAGI, without paids API. TOTALLY FOR FREE 🎁

![AUTO GPT FREE](https://github.com/IntelligenzaArtificiale/Free-AUTO-GPT-with-NO-API/raw/main/img/GH2.gif)

## SETUP REPOSITORY 📂


In [1]:
!git clone https://github.com/IntelligenzaArtificiale/Free-AUTO-GPT-with-NO-API.git


Cloning into 'Free-AUTO-GPT-with-NO-API'...
remote: Enumerating objects: 743, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 743 (delta 53), reused 40 (delta 40), pack-reused 669
Receiving objects: 100% (743/743), 35.38 MiB | 19.51 MiB/s, done.
Resolving deltas: 100% (394/394), done.


In [2]:
import pathlib
import re
import subprocess
import typing


def is_in_jupyter_notebook() -> bool:
    """It checks whether a Jupyter notebook is being run"""
    try:
        get_ipython
        return True
    except NameError:
        return False


def is_on_gcolab() -> bool:
    """It checks whether a Jupyter notebook is being run on Google Colab"""
    if not is_in_jupyter_notebook():
        return False

    return "google.colab" in str(get_ipython())


def is_ubuntu_20_04() -> bool:
    import lsb_release
    metadata = lsb_release.get_os_release()

    distro  = metadata["ID"].lower()
    release = metadata["RELEASE"]

    return distro == "ubuntu" and release == "20.04"


def setup_ubuntu_20_04() -> None:
    """It sets up a Ubuntu 20.04 container with `chromium-browser`

    For more information, see 
    https://github.com/googlecolab/colabtools/issues/3347#issuecomment-1387453484
    """
    # It adds debian buster
    EOF_debian_buster = """\
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
"""
    !echo "$EOF_debian_buster" > /etc/apt/sources.list.d/debian.list

    # It adds keys
    !apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
    !apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
    !apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

    !apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
    !apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
    !apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

    # It adds the debian repo for chromium* packages only
    # Note the double-blank lines between entries
    EOF_chromium_pref = """\
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
"""
    !echo "$EOF_chromium_pref" > /etc/apt/preferences.d/chromium.pref

    # It installs the packages
    !apt-get update
    !apt-get install chromium chromium-driver
    !apt-get install -y xvfb


def setup_requirements() -> None:
    PIP_requirements = " ".join([
        "PyVirtualDisplay", # To run a virtual display
        "undetected-chromedriver==3.2.1",
    ])

    !python3 -m pip install --upgrade $PIP_requirements


def get_py_module_path(module: str) -> typing.Optional[pathlib.Path]:
    """It gets the absolute path of a Python module"""
    r = subprocess.run(
        ["pip", "show", module], 
        capture_output=True
    )

    try:
        r.check_returncode()
    except subprocess.CalledProcessError:
        return None

    stdout = r.stdout.decode()

    try:
        RE_abspath = "\nLocation: (?P<abspath>.*)\n"

        matches = re.search(RE_abspath, stdout)
        abspath = matches.group("abspath")
    except AttributeError:
        return None

    dist_packages = pathlib.Path(abspath).resolve()
    return dist_packages / module


def patch_undetected_chromedriver() -> None:
    """It forces `undetected_chromedriver` to run the Chromium webdriver

    For more information, see 
    https://github.com/ultrafunkamsterdam/undetected-chromedriver/issues/108#issuecomment-1170269377
    """
    chromedriver_filename = "chromedriver_linux64.zip"

    src_chromedriver_filepath = ROOT / chromedriver_filename
    dst_chromedriver_filepath = pathlib.Path("/tmp") / chromedriver_filename

    !zip -j "$src_chromedriver_filepath" /usr/bin/chromedriver

    PY_module = "undetected_chromedriver"
    module_path = get_py_module_path(PY_module)

    patcher_filepath = module_path / "patcher.py"

    with patcher_filepath.open("rt") as f:
        contents = f.read()

    src = f"'file://{src_chromedriver_filepath}'"
    dst = f"'{dst_chromedriver_filepath}'"

    # It is forced to use the local webdriver
    contents = contents.replace(
        f"return urlretrieve(u)[0]",
        f"return urlretrieve({src}, filename={dst})[0]"
    )

    with patcher_filepath.open("wt") as f:
        f.write(contents)


def setup_container() -> None:
    """It sets up the container which is being run"""
    if is_ubuntu_20_04():
        setup_ubuntu_20_04()

    setup_requirements()
    patch_undetected_chromedriver()


ROOT = pathlib.Path("/content")
anchor = ROOT / "anchor.txt"


assert is_on_gcolab(), "It seems you are not on Google Colab"

# It will set the Google Colab container up only
# after disconnections, not after restarts
if not anchor.exists():
    setup_container()
    anchor.touch()

Executing: /tmp/apt-key-gpghome.yczdp74dEy/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.KqJW8VCkTM/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.BUCBCt6xay/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 https://cloud.r-project.org/bin/l

In [3]:
!pip install -r /content/Free-AUTO-GPT-with-NO-API/requirements.txt 
# install dependencies
!pip install PyVirtualDisplay


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.0/727.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 94.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

## HOW TO GET CHATGPT & HUGGINGFACE TOKEN FREE 💡
 


- **HUGGINGFACE🤗** : Visit this simple [official guide](https://huggingface.co/docs/hub/security-tokens)


- **Now Running also with HuggingCHAT**


- **(OPTIONAL BUT BETTER RESULT) CHATGPT🖥** : 
    1. Go to https://chat.openai.com/chat and open the developer tools by `F12`.
    2. Find the `__Secure-next-auth.session-token` cookie in `Application` > `Storage` > `Cookies` > `https://chat.openai.com`.
    3. Copy the value in the `Cookie Value` field.


![image](https://user-images.githubusercontent.com/19218518/206170122-61fbe94f-4b0c-4782-a344-e26ac0d4e2a7.png)


**SAVE only your token, without anything else.**

### 🚀SAVE HERE YOURS TOKENs🚀

**HuggingFACE TOKEN** : _YOURtOkEN_


**CHATGPT TOKEN** : _YOURtOkEN-is-SO-LOnG_

# ⚠️ DONT RUN **CHATGPT** ISTANCE ON COLAB DOSENT WORK FOR NOW ⚠️ 
# ⚠️ USE ONLY HUGGING CHAT ⚠️ 

## RUN Other Agent or Ddevelop your own Agent 👨‍💻 🧑‍💻

In [ ]:
# Check for available AGENT
!ls /content/Free-AUTO-GPT-with-NO-API/OtherAgent/



csvAgent.py  customAgent.py  FreeLLM  pythonAgent.py  startup.csv


In [11]:
# open /content/Free-AUTO-GPT-with-NO-API/OtherAgent/FreeLLM/pyChatGPT.py
# 


!python /content/Free-AUTO-GPT-with-NO-API/OtherAgent/csvAgent.py

#insert this path for csv : /content/Free-AUTO-GPT-with-NO-API/OtherAgent/startup.csv

Select the model you want to use (1 or 2) 
 1) ChatGPT 
 2) HuggingChat 
 >>> 2
Enter the path of the csv file: /content/Free-AUTO-GPT-with-NO-API/OtherAgent/startup.csv
>> START CSV AGENT
> Digit 'exit' for exit or 'your task or question' for start


(Enter your task or question) >> give me the name of the columns .


> Entering new AgentExecutor chain...
Question: give me the name of the columns
Thought: we need to print out all column names so they are visible
Action: df.columns
Action Input: []
Observation: ['R&D Spend', 'Administration', 'Marketing Spend', 'State', 'Profit']
Thought: got the list of column names, next step?
Final Answer: The name of the columns are ['R&D Spend', 'Administration', 'Marketing Spend', 'State', 'Profit'].

> Finished chain.
(Enter your task or question) >> count the number of rows and tell me it .


> Entering new AgentExecutor chain...
Thought: We need to use `.shape` to find out the dimensions of the DataFrame. Action: `print(df.shape)`
Observation:

In [ ]:
!python /content/Free-AUTO-GPT-with-NO-API/OtherAgent/pythonAgent.py

>> STRAT Python AGENT
> Digit 'exit' for exit or 'your task or question' for start


(Enter your task or question) >> Understand, write a single neuron neural network in PyTorch. Take synthetic data for y=2x. Train for 1000 epochs and print every 100 epochs. Return prediction for x = 5


> Entering new AgentExecutor chain...
To write a single neuron neural network in PyTorch, I'll need to define a model that takes in an input x, applies a linear transformation, and returns an output. I'll also need to define a loss function and an optimizer to train the model. Lastly, I'll need to write a loop to train the model for 1000 epochs and print the loss every 100 epochs. After training, I'll use the trained model to make a prediction for x=5.

Action: Python REPL
Action Input:
```python
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define the model
class SingleNeuron(nn.Module):
    def __init__(self):
        super(SingleNeuron, self).__init__()
       

In [ ]:
!pip install -q langchain==0.0.150
!python /content/Free-AUTO-GPT-with-NO-API/OtherAgent/customAgent.py 

>> STRAT Python AGENT
> Digit 'exit' for exit or 'your task or question' for start


(Enter your task or question) >> who is Berlusconi ?


> Entering new AgentExecutor chain...
I think Berlusconi might refer to a person's name. I could try searching for more information about them.
Action: Google search "Berlusconi"
Observation: Silvio Berlusconi is an Italian media tycoon and politician who served as Prime Minister of Italy for nine years.
Thought: Now I know that Berlusconi is a person who was a media tycoon and politician in Italy.
Final Answer: Berlusconi is an Italian media tycoon and politician who served as Prime Minister of Italy for nine years.

> Finished chain.
(Enter your task or question) >> how much cost now Tesla Stock ?


> Entering new AgentExecutor chain...
I think I can use Python to retrieve the current stock price of Tesla. I will need to use a financial API to do this.
Action: Install the `yfinance` library
Action Input: `!pip install yfinance`
Observation: Succe

## RUN BABYAGI 👶

In [ ]:
#Open the /content/Free-AUTO-GPT-with-NO-API/BABYAGI.py and Set "verbose=True" for view the debug 

!python /content/Free-AUTO-GPT-with-NO-API/BABYAGI.py 

Enter the maximum number of iterations: (Suggest from 3 and 5) 5
Enter the objective of the AI system: (Be realistic!) Write me a plan for create a Startup about AI agent capable of performing tasks.

*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list

*****TASK RESULT*****



Sure, here's a todo list for creating a startup about an AI agent capable of performing tasks:

1. Define the problem your AI agent will solve: Identify the specific tasks that your AI agent will be capable of performing, and the target audience who will benefit from it.

2. Research the market: Determine the competition in your target market and understand their strengths and weaknesses.

3. Develop a business plan: Create a plan that outlines your vision, mission, goals, strategies, budget, and revenue model.

4. Build a team: Find experienced professionals who have the skills and expertise to develop and market your AI agent.

5. Create a prototype: Develop a working prototype o

## RUN AUTOGPT 🤖

In [ ]:
#Open the /content/Free-AUTO-GPT-with-NO-API/BABYAGI.py and modify or add new tool 
"""
#ACTUAL TOOL 
tools = [
    web_search, #DuckDuckGoSearchRun()
    WriteFileTool(root_dir="/data"),
    ReadFileTool(root_dir="/data"),
    process_csv,
    query_website_tool, playwright()
    # HumanInputRun(), # Activate if you want the permit asking for help from the human
]
"""
!python /content/Free-AUTO-GPT-with-NO-API/AUTOGPT.py

Enter the objective of the AI system: (Be realistic!) Write a small report on the current temperature in New York, including the time
{
    "thoughts": {
        "text": "I need to determine the current temperature in New York. I can use a search engine to find this information.",
        "reasoning": "I will use the DuckDuckGo Search command to search for the current temperature in New York. This will provide me with up-to-date information on the current temperature in the city.",
        "plan": "- Use the DuckDuckGo Search command to search for the current temperature in New York\n- Save the result to a file",
        "criticism": "I need to make sure that the information I find is accurate and up-to-date. I should also make sure that the file is saved in a place where it can be easily retrieved later.",
        "speak": "I will use the DuckDuckGo Search command to find the current temperature in New York."
    },
    "command": {
        "name": "DuckDuckGo Search",
        "args":

## RUN CAMEL with Streamlit.io 🐪

In [ ]:
# SETUP COLAB for run Streamlit
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.069s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [ ]:
!streamlit run /content/Free-AUTO-GPT-with-NO-API/Camel.py &>/content/logs.txt &
!npx localtunnel --port 8501

npx: installed 22 in 3.929s
your url is: https://seven-pans-add-35-237-170-21.loca.lt
^C


## RUN Meta-Prompt 👥

In [ ]:
!python /content/Free-AUTO-GPT-with-NO-API/MetaPrompt.py

Enter the objective of the AI system: (Be realistic!) write a detalied plan to solve world hunger
Enter the maximum number of interactions per episode: 2
Enter the maximum number of episodes: 2
[Episode 1/2]


> Entering new LLMChain chain...
Prompt after formatting:

    Instructions: None
    
    Human: write a detalied plan to solve world hunger
    Assistant:

> Finished chain.
(Step 1/2)
Assistant: Ending world hunger is a complex and challenging problem that requires a comprehensive and multi-faceted approach. Here is a detailed plan that could be used to tackle world hunger:

1. Increase agricultural productivity: Increasing agricultural productivity in developing countries is crucial to reducing hunger. This can be done by providing farmers with better seeds, fertilizers, and tools, and by improving irrigation systems. In addition, support for research and development of sustainable agricultural practices can help to increase yields and reduce waste.

2. Improve access to mark